## 📌 Note on Reproducibility and Scope

This notebook serves as an **open-source, self-contained illustration** of the **model-compression techniques** explored in our **HMPL final project**, including **knowledge distillation, quantization, and pruning**.

Due to the use of **proprietary Google infrastructure** in the original experiments, the full production pipeline cannot be released publicly. Instead, we provide a **faithful synthetic reproduction** of the training and compression workflow that mirrors the **core modeling ideas, loss formulations, and optimization strategies** used in the original project.

This approach ensures **transparency, clarity, and reproducibility**, while preserving the conceptual and technical integrity of the original system.


# Efficient Model Compression Pipeline  
## Google Ads Conversion Prediction Example

### 1. Teacher and Student Model Definition

In Google Ads’ internal **conversion value prediction system**, a large **teacher model** predicts conversion outcomes using a **multi-head architecture**:

- **Conversion probability head (classification)**  
- **Conversion value / count head (regression)**  

The teacher model is trained with a **combination of losses**:

- **Binary Cross-Entropy (sigmoid) loss** for conversion probability  
- **Poisson negative log-likelihood loss** for conversion count or value  

The Poisson log loss (also known as *negative Poisson log-likelihood*) is well suited for modeling **count data** such as conversions and serves as the **primary objective** in Google’s system.

---

### Student Model

The **student model** is a smaller, more efficient neural network designed to **mimic the teacher’s behavior**. It uses the same two-output structure:

- Conversion probability  
- Conversion value  

but with a simpler architecture. Through **knowledge distillation**, the student learns to approximate the teacher’s predictions while using significantly fewer parameters, enabling **faster inference and lower serving costs** in production.

---

### Synthetic Experimental Setup

To illustrate this pipeline, we define a **synthetic conversion prediction task**:

- Each training sample consists of:
  - A random feature vector
  - A binary label indicating whether a conversion occurred
  - A non-negative conversion count (with many zeros, reflecting real-world sparsity)

---

### Model Architectures

**Teacher Model**
- Architecture: [64, 32] - Two dense hidden layers
- Two output heads:
  - `conv_prob`: Sigmoid activation (conversion probability)
  - `conv_value`: Softplus activation (non-negative count for Poisson loss)
  - `prob_logits`: Raw logits for distillation

**Student Model**
- Architecture: [32] - single hidden layer (fixed to ensure dimension compatibility with the teacher's bottleneck layer)
- Same two output heads and activations as the teacher
- Optimized for efficiency while maintaining predictive performance

---
### Hyperparameter Sweep Configuration

We include an example config with sweeps to show how the experiments can pull from the config and sweep over the specified hyperparamters. This example is very simple compared to the more sophisticated pyplan config options at Google, but demonstrates the high-level workflow.


This setup demonstrates how **model compression via teacher–student training** can retain accuracy while significantly improving deployment efficiency in large-scale systems such as Google Ads.


In [3]:
import numpy as np
import tensorflow as tf
import itertools

# example of user config input (in reality would be separate file)
HYPERPARAM_CONFIG = {
    "student": {
        "dropout_rate": [0.0, 0.1, 0.2]
    },
    "distillation": {
        "alpha": [0.1, 0.5, 0.9],
        "temperature": [3.0, 5.0]
    }
}

# Fix random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Synthetic data dimensions
num_features = 10
num_samples = 10000

# Generate synthetic feature matrix
X_train = np.random.normal(size=(num_samples, num_features)).astype(np.float32)

# Generate synthetic conversion labels:
# We simulate a scenario where conversion count follows a Poisson distribution based on features.
# Define an arbitrary "true" weight vector for Poisson rate:
true_w = np.random.normal(scale=0.5, size=(num_features, 1))
# Compute a latent log-rate and apply exp to get Poisson lambda
log_lambda = X_train.dot(true_w)  # shape (num_samples, 1)
lambda_vals = np.exp(log_lambda).flatten()
# Sample conversion counts from Poisson(lambda). (Clip to some max for safety)
y_count = np.random.poisson(lam=lambda_vals).astype(np.float32)
y_count = np.clip(y_count, 0, 20)  # limit extreme values for stability

# Conversion occurred or not (binary label) – 1 if count > 0 else 0
y_conv = (y_count > 0).astype(np.float32)

# Prepare targets for Keras
y_train = {
    "conv_prob": y_conv,
    "conv_value": y_count,
    "prob_logits": np.zeros_like(y_conv)
}

def build_model(hidden_units, dropout=0.0, name="Model"):
    inputs = tf.keras.Input(shape=(10,), name="features")
    x = inputs

    if isinstance(hidden_units, int):
        hidden_units = [hidden_units]

    for i, units in enumerate(hidden_units):
        layer_name = "bottleneck" if i == len(hidden_units) - 1 else f"hidden_{i}"
        x = tf.keras.layers.Dense(units, activation='relu', name=layer_name)(x)
        if dropout > 0:
            x = tf.keras.layers.Dropout(dropout)(x)

    # Outputs
    prob_logits = tf.keras.layers.Dense(1, activation=None, name="prob_logits")(x)
    prob_output = tf.keras.layers.Activation('sigmoid', name="conv_prob")(prob_logits)
    value_output = tf.keras.layers.Dense(1, activation=tf.nn.softplus, name="conv_value")(x)

    # FIX: Return a Dictionary of outputs to match the keys in loss/y_train
    return tf.keras.Model(
        inputs=inputs,
        outputs={
            "conv_prob": prob_output,
            "conv_value": value_output,
            "prob_logits": prob_logits
        },
        name=name
    )

### Explanation of the Synthetic Training Setup and Losses

In this code, we generate **random input features** and construct labels to mimic a realistic conversion prediction task:

- The **conversion count** `y_count` is sampled from a Poisson distribution:  
  $$
  y_{\text{count}} \sim \text{Poisson}(\exp(w \cdot x))
  $$
- The **binary conversion label** `y_conv` is defined as:
  $$
  y_{\text{conv}} = \mathbb{1}(y_{\text{count}} > 0)
  $$

---

### Teacher Model Training Objective

The teacher model is trained using **two loss functions**, corresponding to its two output heads:

- **Binary Cross-Entropy Loss** for the `conv_prob` output, modeling the probability of a conversion
- **Poisson Loss** (`tf.keras.losses.Poisson`) for the `conv_value` output, modeling the conversion count or value

The Poisson loss in TensorFlow implements the **negative Poisson log-likelihood**:
$$
L = y_{\text{pred}} - y_{\text{true}} \log(y_{\text{pred}})
$$
which matches the standard Poisson log loss **up to constant terms** that do not affect optimization.

---

### Training Context

For demonstration purposes, the teacher model is trained **briefly on synthetic data**. In a real production setting, the teacher would be **pre-trained on a massive dataset** with richer features and longer training schedules.

With the teacher model trained, we now proceed to **compress the model using knowledge distillation**, transferring its predictive behavior to a smaller and more efficient student network.


## 2. Knowledge Distillation Training (Hard + Soft Targets)

**Knowledge distillation (KD)** compresses a large **teacher** model into a smaller **student** by training the student on both:
- **Hard targets** (ground-truth labels), and  
- **Soft targets** (the teacher’s predictions).

The key idea is that the teacher’s outputs encode *dark knowledge* about the underlying function, helping the student generalize better than training on labels alone.

---

### Distillation Loss

The student is trained with a **weighted combination** of losses:
$$
L_{\text{total}} = \alpha \, L_{\text{hard}}(y_{\text{true}}, y_{\text{student}})
+ \beta \, L_{\text{soft}}(y_{\text{teacher}}, y_{\text{student}})
$$
where \(\alpha\) and \(\beta\) balance supervision from true labels and the teacher (often \(\alpha + \beta = 1\)).

- **Hard losses**:
  - Binary cross-entropy for conversion probability  
  - Poisson loss for conversion count  
- **Soft losses**:
  - Mean squared error (MSE) between teacher and student outputs for each head  

---

### Optional Feature Matching

We also include an optional **feature-based distillation** term (FitNets-style), where an intermediate student representation is trained to match a corresponding **teacher “hint” layer**. This additional supervision can further improve student performance.

---

### Training Procedure

The teacher’s weights are **frozen**, and the student is trained using the combined distillation loss. Below, we illustrate this with a **custom training loop**.


In [5]:
def run_experiment(dropout, alpha, temp, X_train, y_train, teacher_weights):

    # 1. Build Student
    student_model = build_model([32], dropout=dropout, name="Student")

    # 2. Build Teacher
    teacher_model = build_model([64, 32], name="Teacher")
    teacher_model.set_weights(teacher_weights)
    teacher_model.trainable = False

    # 3. Feature Extractors
    teacher_feat_extractor = tf.keras.Model(inputs=teacher_model.inputs,
                                            outputs=teacher_model.get_layer("bottleneck").output)
    student_feat_extractor = tf.keras.Model(inputs=student_model.inputs,
                                            outputs=student_model.get_layer("bottleneck").output)

    # 4. Training Loop
    batch_size = 64
    epochs = 2
    learning_rate = 0.001
    gamma = 0.1

    optimizer = tf.keras.optimizers.Adam(learning_rate)
    dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size)

    final_loss = 0.0

    for epoch in range(epochs):
        epoch_loss = 0.0
        steps = 0
        for x_batch, y_batch in dataset:
            # Teacher inference
            t_preds = teacher_model(x_batch, training=False)
            t_prob, t_val, t_logits = t_preds["conv_prob"], t_preds["conv_value"], t_preds["prob_logits"]

            with tf.GradientTape() as tape:
                # Student training (dictionary output)
                s_preds = student_model(x_batch, training=True)
                s_prob = s_preds["conv_prob"]
                s_val = s_preds["conv_value"]
                s_logits = s_preds["prob_logits"]

                # FIX: Squeeze outputs to match label dimensions (64,) instead of (64, 1)
                s_prob_squeezed = tf.squeeze(s_prob, axis=-1)
                s_val_squeezed = tf.squeeze(s_val, axis=-1)

                # Hard Loss (compare student output to Ground Truth labels)
                loss_hard = tf.reduce_mean(tf.keras.losses.binary_crossentropy(y_batch["conv_prob"], s_prob_squeezed)) + \
                            tf.reduce_mean(tf.keras.losses.Poisson()(y_batch["conv_value"], s_val_squeezed))

                # Soft Loss (compare student logits to Teacher logits)
                t_prob_soft = tf.nn.sigmoid(t_logits / temp)
                s_prob_soft = tf.nn.sigmoid(s_logits / temp)
                # FIX: Instantiate the loss class, then call it
                mse = tf.keras.losses.MeanSquaredError()
                loss_soft = tf.reduce_mean(mse(t_prob_soft, s_prob_soft)) + \
                            tf.reduce_mean(mse(t_val, s_val))

                # Feature Loss
                feat_loss = 0.0
                if gamma > 0:
                    t_feat = teacher_feat_extractor(x_batch, training=False)
                    s_feat = student_feat_extractor(x_batch, training=True)
                    feat_loss = tf.reduce_mean(mse(t_feat, s_feat))

                total_loss = (1 - alpha) * loss_hard + (alpha * (temp**2) * loss_soft) + (gamma * feat_loss)

            grads = tape.gradient(total_loss, student_model.trainable_weights)
            optimizer.apply_gradients(zip(grads, student_model.trainable_weights))
            epoch_loss += total_loss
            steps += 1

        final_loss = float(epoch_loss / steps)

    return final_loss


def generate_configs():
    dropout_list = HYPERPARAM_CONFIG['student']['dropout_rate']
    alpha_list = HYPERPARAM_CONFIG['distillation']['alpha']
    temp_list = HYPERPARAM_CONFIG['distillation']['temperature']

    for dropout, alpha, temp in itertools.product(dropout_list, alpha_list, temp_list):
        label = f"Dropout={dropout}, Alpha={alpha}, Temp={temp}"
        yield (dropout, alpha, temp), label


## train teacher (stays constant)
print("Training Base Teacher ([64, 32])...")
base_teacher = build_model([64, 32], name="Teacher")
# Using Dictionary Loss to match the new Dictionary Output
base_teacher.compile(optimizer='adam',
                      loss={"conv_prob": "binary_crossentropy",
                            "conv_value": "poisson",
                            "prob_logits": None})
base_teacher.fit(X_train, y_train, epochs=5, batch_size=64, verbose=0)
teacher_weights = base_teacher.get_weights()
print("Teacher Ready.\n")


## run experiments with all sweep combinations
results = []
print(f"{'RUN CONFIGURATION':<30} | {'LOSS':<8}")
print("-" * 50)

for (dropout, alpha, temp), label in generate_configs():
    loss = run_experiment(dropout, alpha, temp, X_train, y_train, teacher_weights)
    print(f"{label:<50} | {loss:.4f}")
    results.append((label, loss))

print("-" * 50)
best_run = min(results, key=lambda x: x[1])
print(f"Best Configuration: {best_run[0]}")
print(f"Lowest Loss: {best_run[1]:.4f}")


Training Base Teacher ([64, 32])...
Teacher Ready.

RUN CONFIGURATION              | LOSS    
--------------------------------------------------
Dropout=0.0, Alpha=0.1, Temp=3.0                   | 1.9028
Dropout=0.0, Alpha=0.1, Temp=5.0                   | 10.2212
Dropout=0.0, Alpha=0.5, Temp=3.0                   | 15.9344
Dropout=0.0, Alpha=0.5, Temp=5.0                   | 39.7678
Dropout=0.0, Alpha=0.9, Temp=3.0                   | 33.9107
Dropout=0.0, Alpha=0.9, Temp=5.0                   | 65.6466
Dropout=0.1, Alpha=0.1, Temp=3.0                   | 3.1110
Dropout=0.1, Alpha=0.1, Temp=5.0                   | 10.1808
Dropout=0.1, Alpha=0.5, Temp=3.0                   | 15.9606
Dropout=0.1, Alpha=0.5, Temp=5.0                   | 62.8417
Dropout=0.1, Alpha=0.9, Temp=3.0                   | 31.7078
Dropout=0.1, Alpha=0.9, Temp=5.0                   | 78.9286
Dropout=0.2, Alpha=0.1, Temp=3.0                   | 3.5239
Dropout=0.2, Alpha=0.1, Temp=5.0                   | 9.6949
Dropo

### Distillation Training Loop Summary

In each training batch, we compute:

- **Teacher predictions**: Conversion probability and value from the frozen teacher (soft targets).
- **Student predictions**: Corresponding outputs from the student model.

---

### Loss Components

- **Hard loss**:  
  - Binary cross-entropy between true conversion labels and student probabilities  
  - Poisson loss between true counts and student predicted values  

- **Soft loss**:  
  - Mean squared error (MSE) between teacher and student outputs for both heads  

- **Feature matching loss (optional)**:  
  - MSE between intermediate teacher (“hint”) and student (“guided”) representations, encouraging the student to mimic the teacher’s internal features  
  - Scaled by a factor \(\gamma\) (set to 0 to disable)

---

### Optimization

The **total student loss** is a weighted sum of hard loss, soft loss, and feature matching loss. In this example:
- \(\alpha = 0.5\) balances hard vs. soft losses  
- \(\gamma = 0.1\) lightly weights feature matching  

The student is trained for a few epochs, during which the distillation loss decreases, showing that it is learning from both ground truth and teacher outputs.

---

At this stage, the student model closely approximates the teacher. Next, we apply **additional compression techniques** such as quantization and pruning to further reduce model size and inference cost.


## 3. Post-Training Quantization (PTQ)

**Post-training quantization (PTQ)** reduces model size and improves inference efficiency *after training*, without updating model weights. Typically, weights (and optionally activations) are converted from **float32 to int8**, often achieving up to **~4× model size reduction** with minimal accuracy loss.

---

### PTQ with TensorFlow Lite

PTQ is applied using the **TensorFlow Lite converter**, which can directly convert a trained `tf.keras` model.

To quantize **activations** (not just weights), the converter requires a **representative dataset** for calibration. This dataset is used to estimate activation ranges (min/max) and compute appropriate quantization scales and zero-points.

---

### Setup

Here, we apply PTQ to the **trained student model**:
- Convert the Keras model to **TFLite int8 format**
- Use a small subset of the training data as the **representative dataset** for calibration

This produces a compact, efficient model suitable for low-latency, production inference.


In [ ]:
# 1. Convert the trained student model to a float32 TFLite model (for size comparison)
converter_fp32 = tf.lite.TFLiteConverter.from_keras_model(student_model)
tflite_model_fp32 = converter_fp32.convert()

# 2. Convert the student model to an int8 quantized TFLite model (post-training quantization)
converter_int8 = tf.lite.TFLiteConverter.from_keras_model(student_model)
converter_int8.optimizations = [tf.lite.Optimize.DEFAULT]  # enable default optimizations (which include quantization)
# Provide a representative dataset generator for full integer quantization
def representative_data_gen():
    for i in range(100):
        # Take random batches from training data
        sample = X_train[i:i+1]
        yield [sample.astype(np.float32)]
converter_int8.representative_dataset = representative_data_gen
# Set the supported ops and ensure INT8 quantization for inputs/outputs
converter_int8.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter_int8.inference_input_type = tf.int8
converter_int8.inference_output_type = tf.int8

tflite_model_int8 = converter_int8.convert()

# Save TFLite models to disk to compare sizes
with open("student_model_fp32.tflite", "wb") as f:
    f.write(tflite_model_fp32)
with open("student_model_int8.tflite", "wb") as f:
    f.write(tflite_model_int8)

# Compare file sizes
import os
fp32_size = os.path.getsize("student_model_fp32.tflite") / 1024
int8_size = os.path.getsize("student_model_int8.tflite") / 1024
print(f"FP32 TFLite model size: {fp32_size:.2f} KB")
print(f"INT8 TFLite model size: {int8_size:.2f} KB")


### PTQ Results and Validation

After conversion, we compare the **file sizes** of the float32 and int8 TFLite models. As expected, the **int8 quantized model** is approximately **4× smaller**, since weights are stored in 8-bit rather than 32-bit precision.

Post-training quantization does **not require retraining**, making it simple to apply. The main trade-off is a potential **small accuracy drop** due to quantization error in weights and activations. In practice, well-calibrated quantized models typically retain accuracy close to the float baseline for many tasks.

Finally, we validate the quantized model by running inference with a **TensorFlow Lite interpreter** to confirm it produces reasonable outputs.


In [ ]:
# Verify the quantized model on a couple of examples
interpreter = tf.lite.Interpreter(model_content=tflite_model_int8)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test on a few samples
for i in range(3):
    x = X_train[i:i+1]
    # Quantize the input from float32 to int8
    input_scale, input_zero_point = input_details[0]['quantization']
    x_int8 = (x / input_scale + input_zero_point).astype(np.int8)
    interpreter.set_tensor(input_details[0]['index'], x_int8)
    interpreter.invoke()
    # Dequantize output back to float32 for interpretation
    output_prob = interpreter.get_tensor(output_details[0]['index'])
    output_value = interpreter.get_tensor(output_details[1]['index'])
    print(f"Example {i}: conv_prob={output_prob[0][0]:.3f}, conv_value={output_value[0][0]:.3f}")


This step runs the **quantized student model** on sample inputs and prints the predicted **conversion probability** and **conversion value**. Comparing these outputs with the original float32 model shows that predictions remain **very close**, differing only by small quantization rounding errors.

---

### Summary

Post-training quantization significantly improves efficiency by making the student model **much smaller and faster**, typically achieving a **~4× reduction in model size** with int8 weights. This makes PTQ a simple and effective optimization for production deployment.

If the small accuracy loss from quantization is unacceptable, the next step would be to apply **quantization-aware training (QAT)**.


## 4. Quantization-Aware Training (QAT)

**Quantization-aware training (QAT)** simulates low-bit (e.g., int8) quantization *during training* by inserting **fake quantization operations** for weights and activations. This allows the model to **adapt to quantization effects** and preserve accuracy after conversion.

Compared to PTQ, QAT typically achieves **better final accuracy**, at the cost of **additional fine-tuning time**.

---

### QAT with TensorFlow Model Optimization Toolkit

We apply QAT to the **student model** using the TensorFlow Model Optimization Toolkit (TF-MOT):

- The student model is wrapped with  
  `tfmot.quantization.keras.quantize_model`
- Fake quantization layers are inserted (visible as `quant_` layers in the model summary)
- The quantization-aware model is **fine-tuned for a few epochs**
- The trained model is then converted to a **TFLite int8 model**

This produces a quantized model that is both **compact and more accurate** than PTQ alone, making it suitable for accuracy-sensitive production deployments.


In [ ]:
!pip install -q tensorflow-model-optimization

import tensorflow_model_optimization as tfmot

# Create a quantization-aware model from the student model
quantize_model = tfmot.quantization.keras.quantize_model
q_aware_student = quantize_model(student_model)  # this wraps the student model

# Compile the QAT model with the same losses as the original (two outputs)
q_aware_student.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                        loss={"conv_prob": "binary_crossentropy", "conv_value": tf.keras.losses.Poisson()})

q_aware_student.summary()  # to show quantized layers (optional)


In [ ]:
# Fine-tune the quantization-aware model on a subset of data (to simulate QAT)
X_subset = X_train[:2000]
y_subset = { "conv_prob": y_conv[:2000].astype(np.float32), "conv_value": y_count[:2000].astype(np.float32) }

# Train for a couple of epochs
q_aware_student.fit(X_subset, y_subset, batch_size=64, epochs=3, verbose=0)


In [ ]:
# Convert the quantization-aware trained model to a fully quantized TFLite model
converter_qat = tf.lite.TFLiteConverter.from_keras_model(q_aware_student)
converter_qat.optimizations = [tf.lite.Optimize.DEFAULT]
# (No representative dataset needed, as ranges are already determined by QAT)
tflite_model_qat = converter_qat.convert()

with open("student_model_qat_int8.tflite", "wb") as f:
    f.write(tflite_model_qat)

qat_int8_size = os.path.getsize("student_model_qat_int8.tflite") / 1024
print(f"QAT INT8 TFLite model size: {qat_int8_size:.2f} KB")


### QAT Results and Comparison

The **QAT-generated int8 model** has a file size similar to the PTQ int8 model, since both are fully quantized to 8-bit. The key advantage of QAT is **accuracy**, not size.

By training with simulated quantization effects, the QAT model typically achieves accuracy **much closer to the original float model** than PTQ, especially for more complex or sensitive tasks. In many production settings, the accuracy drop with QAT is **negligible or eliminated**.

We can validate this by evaluating the QAT model on a test set or comparing its predictions against the float model. In this simplified example, differences may be small due to the toy setup, but in real systems QAT is often essential when PTQ accuracy loss is unacceptable.

---

**Recap:** QAT integrates quantization into training using TF-MOT, followed by conversion to a fully int8 TFLite model. The result is a compact model with **~4× size reduction** and accuracy that is often within **~1% (or equal)** to the float baseline.


## 5. Model Pruning for Weight Sparsity

**Pruning** reduces model size by removing low-impact weights, making the network sparse. Many neural network weights contribute little to predictions, so zeroing them out and retraining can significantly reduce model size with minimal accuracy loss. In practice, pruning can yield **~3× size reduction**, and when combined with quantization, up to **~10× smaller models**.

---

### Pruning Workflow

1. **Sensitivity Analysis**  
   Test different sparsity levels (e.g., 10–90%) for individual layers or the full model and measure performance degradation. This helps identify which layers are more or less sensitive to pruning.

2. **Pruning + Fine-Tuning**  
   Gradually prune weights to a target sparsity and **fine-tune the model**, typically with a smaller learning rate, to recover lost accuracy.

---

### Demonstration Plan

We first perform a **simple layer-wise sensitivity test** by pruning a fixed percentage of weights in each dense layer of the student model and measuring the change in validation loss. Based on this, we then apply **structured pruning** using the TensorFlow Model Optimization Toolkit (TF-MOT) to sparsify the student model while maintaining performance.


In [ ]:
# Create a validation/test set for evaluation
X_val = X_train[-1000:]
y_val = { "conv_prob": y_conv[-1000:].astype(np.float32), "conv_value": y_count[-1000:].astype(np.float32) }

# Compile the student model for evaluation (using the same losses)
student_model.compile(loss={"conv_prob": "binary_crossentropy", "conv_value": tf.keras.losses.Poisson()})

# Evaluate baseline performance
baseline_loss = student_model.evaluate(X_val, y_val, verbose=0)
print(f"Baseline model loss (total): {baseline_loss:.4f}")

# Sensitivity analysis: prune 50% weights in each Dense layer and measure loss
for layer in student_model.layers:
    if isinstance(layer, tf.keras.layers.Dense):
        # Skip output layers for pruning test (focus on hidden layers)
        if layer.name in ["conv_prob", "conv_value"]:
            continue
        original_weights = layer.get_weights()  # [kernel, bias]
        if not original_weights:  # if layer has no weights (shouldn’t happen for Dense)
            continue
        W, b = original_weights
        # Determine threshold for 50% sparsity (median of absolute weights)
        threshold = np.percentile(np.abs(W), 50)
        W_pruned = np.where(np.abs(W) < threshold, 0, W)
        layer.set_weights([W_pruned, b])
        loss_after = student_model.evaluate(X_val, y_val, verbose=0)
        print(f"After pruning 50% of weights in layer '{layer.name}': loss = {loss_after:.4f}")
        # Restore original weights
        layer.set_weights(original_weights)


### Layer-Wise Pruning Sensitivity Analysis

This code iterates over each **hidden Dense layer** of the student model and performs a simple pruning test:

- Computes the **50th percentile** of absolute weight values in the layer  
- Sets weights below this threshold to zero, creating ~**50% sparsity**
- Evaluates the model loss on a **validation set** with the layer pruned
- Restores the original weights before moving to the next layer

By comparing the printed losses, we can identify which layers are **most sensitive to pruning**—larger loss increases indicate layers that should be pruned more conservatively.

In practice, this analysis would be extended to multiple sparsity levels (e.g., 10%–90%) to determine the **maximum acceptable pruning** per layer (such as keeping loss degradation under ~2%).

Next, we apply **gradual pruning** using TensorFlow Model Optimization Toolkit’s pruning API, which incrementally increases sparsity during fine-tuning and typically preserves accuracy better than one-shot pruning.


In [ ]:
import tensorflow_model_optimization as tfmot

# Set up pruning parameters: gradually prune to 80% sparsity over 5 epochs
pruning_epochs = 5
steps_per_epoch = int(np.ceil(X_train.shape[0] / 64))
prune_schedule = tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.0,
                                                      final_sparsity=0.8,
                                                      begin_step=0,
                                                      end_step=pruning_epochs * steps_per_epoch)

pruning_wrapper = tfmot.sparsity.keras.prune_low_magnitude
pruned_model = pruning_wrapper(student_model, pruning_schedule=prune_schedule)

# Compile the pruned model (use same loss and optimizer)
pruned_model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                     loss={"conv_prob": "binary_crossentropy", "conv_value": tf.keras.losses.Poisson()})

# Fine-tune the model with pruning
callback = tfmot.sparsity.keras.UpdatePruningStep()  # updates pruning step each epoch
pruned_model.fit(X_train, y_train, batch_size=64, epochs=pruning_epochs, callbacks=[callback], verbose=0)

# Strip pruning wrappers to get a normal model with pruned weights
pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

# Check final sparsity
total_weights = 0
zeros = 0
for weight in pruned_model.get_weights():
    total_weights += weight.size
    zeros += np.sum(weight == 0)
print(f"Final model sparsity: {zeros/total_weights:.2%} of weights are zero.")


### Gradual Pruning and Results

This code applies **gradual magnitude pruning** using a polynomial decay schedule:

- Sparsity increases smoothly from **0% to 80%** over several epochs, allowing the model to adapt as weights are removed.
- `prune_low_magnitude` wraps the model layers with pruning logic, and the `UpdatePruningStep` callback updates sparsity during training.
- After fine-tuning, `strip_pruning` removes pruning wrappers, leaving a standard Keras model with many weights set to zero.

The final sparsity check confirms that the model reaches **~80% zero weights** (minor deviations are expected since biases are typically not pruned).

---

### Evaluation and Extensions

The pruned model should be evaluated to ensure performance remains acceptable. If accuracy drops too much, the target sparsity or pruning schedule can be adjusted, or the model can be further fine-tuned at a lower learning rate.

**Notes:**
- Pruning can be **combined with quantization** (e.g., PTQ or QAT) for much higher compression, often up to **~10× size reduction**.
- This example uses **unstructured pruning** (individual weights). Hardware speedups depend on sparse execution support; structured pruning can yield clearer speed gains but may affect accuracy more.
- Adding **L1 regularization** during initial training can improve prune-ability, though it was not used here.


## 6. Conclusion

We presented an end-to-end **model compression pipeline** inspired by Google Ads conversion prediction. Starting from a large **teacher model** with multi-task outputs (conversion probability and value, optimized with sigmoid and Poisson losses), we trained a compact **student model** using knowledge distillation.

The student was further optimized using:
- **Post-training quantization (PTQ)** for ~4× model size reduction with minimal accuracy loss  
- **Quantization-aware training (QAT)** to better preserve accuracy under int8 quantization  
- **Weight pruning**, including sensitivity analysis and gradual pruning to ~80% sparsity with fine-tuning  

Together, these steps significantly reduce **model size, latency, and compute cost** while maintaining strong performance. This mirrors common large-scale production practices: train a strong model, distill it, and deploy a **small, efficient, and accurate** version suitable for real-world inference environments such as web services or mobile devices.
